In [ ]:
import panel_highcharts as ph

ph.config.theme("auto")

In [ ]:
import panel as pn

# 注意有些文件引用其他文件，所以顺序也很重要
ph.config.highstock_js_files(others=["jquery-1.8.3","highcharts/modules/drag-panes","highcharts/indicators/indicators","highcharts/indicators/volume-by-price",])

pn.extension('highstock')

In [ ]:
import urllib, requests
from pyjson5 import decode

system_proxies = urllib.request.getproxies()

url = "https://data.jianshukeji.com/json/aapl-ohlcv.json"
if system_proxies and 'http' in system_proxies:
	data = requests.get(url, proxies={"https": system_proxies['http']})
else:
	data = requests.get(url)
data = decode(data.text)
# split the data set into ohlc and volume
ohlc = []
volume = []
dataLength = len(data)
groupingUnits = [
		['week', [1]],
		['month', [1, 2, 3, 4, 6]]
]
i = 0
for i in range(len(data)):
	ohlc.append([
		data[i][0], # the date
		data[i][1], # open
		data[i][2], # high
		data[i][3], # low
		data[i][4] # close
	])
	volume.append([
		data[i][0],  # the date
		data[i][5]  # the volume
	])
ohlc
volume

In [ ]:
configuration_str = """{
		rangeSelector: {
			selected: 2
		},
		title: {
			text: 'AAPL Historical'
		},
		subtitle: {
			text: 'With SMA and Volume by Price technical indicators'
		},
		yAxis: [{
			startOnTick: false,
			endOnTick: false,
			labels: {
				align: 'right',
				x: -3
			},
			title: {
				text: 'OHLC'
			},
			height: '60%',
			lineWidth: 2,
			resize: {
				enabled: true
			}
		}, {
			labels: {
				align: 'right',
				x: -3
			},
			title: {
				text: 'Volume'
			},
			top: '65%',
			height: '35%',
			offset: 0,
			lineWidth: 2
		}],
		tooltip: {
			split: true
		},
		plotOptions: {
			series: {
				dataGrouping: {
					units: "groupingUnits"
				}
			}
		},
		series: [{
			type: 'candlestick',
			name: 'AAPL',
			id: 'aapl',
			zIndex: 2,
			data: "ohlc"
		}, {
			type: 'column',
			name: 'Volume',
			id: 'volume',
			data: "volume",
			yAxis: 1
		}, {
			type: 'vbp',
			linkedTo: 'aapl',
			params: {
				volumeSeriesID: 'volume'
			},
			dataLabels: {
				enabled: false
			},
			zoneLines: {
				enabled: false
			}
		}, {
			type: 'sma',
			linkedTo: 'aapl',
			zIndex: 1,
			marker: {
				enabled: false
			}
		}]
	}"""
configuration = decode(configuration_str)
configuration["plotOptions"]["series"]["dataGrouping"]["units"] = groupingUnits
configuration["series"][0]["data"] = ohlc
configuration["series"][1]["data"] = volume

configuration

In [ ]:
chart = ph.HighStock(object=configuration, sizing_mode="stretch_both", min_height=600)
chart.servable()